In [ ]:
import pluto_sdr_pr.processing

from scipy.signal import hilbert

import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
duration = 1
fs = 1000
samples = int(fs*duration)
t = np.arange(samples) / fs
pulse_on_duration = 50
pulse_off_duration = samples - pulse_on_duration
time_offset = 250
freq_offset = 90

waveform = np.concatenate([np.sin(t[:pulse_on_duration] * 100 * 2 * np.pi), np.zeros(pulse_off_duration)])
ref_waveform = waveform.copy()
ref_waveform += np.random.default_rng(seed=1337).standard_normal(samples) * 0.2
surv_waveform = 0.3 * np.roll(np.real(waveform.copy() * np.exp(-2j * t * np.pi * freq_offset)), time_offset)
surv_waveform += ref_waveform * 0.5
surv_waveform += np.random.default_rng(seed=1337).standard_normal(samples) * 0.01

ref_signal = hilbert(ref_waveform)
surv_signal = hilbert(surv_waveform)


amb_pre_clean = pluto_sdr_pr.processing.fast_ambiguity(
    samples,
    fs,
    ref_signal,
    surv_signal,
)
abs_amb_pre_clean = np.abs(amb_pre_clean)
peak_pre_clean = np.unravel_index(
    np.argmax(abs_amb_pre_clean),
    abs_amb_pre_clean.shape
)

fig, (ax0, ax1, ax2, ax3) = plt.subplots(4, figsize=(11.69, 8.27))
fig.suptitle("Reference and Echo Signals as Recorded by Receiver", fontsize=16)

ax0.plot(waveform)
ax0.set_xlabel("samples")
ax0.set_xlim([0, t.shape[0]])
ax0.set_ylim([-1, 1])
ax0.set_title("Sent Waveform")

ax1.plot(ref_waveform / np.abs(ref_waveform).max())
ax1.set_xlabel("samples")
ax1.set_xlim([0, t.shape[0]])
ax1.set_ylim([-1, 1])
ax1.set_title("Noisy Reference Receiver Channel")

ax2.plot(surv_waveform / np.abs(ref_waveform).max())
ax2.set_xlabel("samples")
ax2.set_xlim([0, t.shape[0]])
ax2.set_ylim([-1, 1])
ax2.set_title("Noisy Surveillance Receiver Channel")
ax2.axvspan(0, pulse_on_duration, facecolor='red', alpha=.2)
ax2.axvspan(time_offset, time_offset + pulse_on_duration, facecolor='blue', alpha=.2)

ax3.imshow(abs_amb_pre_clean.T, interpolation="nearest", aspect="auto", cmap="Greys")
ax3.set_yticks(np.linspace(0, fs, 9, endpoint=True))
ax3.set_yticklabels(
    map(lambda y: f"{-(y - fs // 2):.0f}", ax3.get_yticks())
)
ax3.set_xlabel("range")
ax3.set_ylabel("doppler")
ax3.set_xlim([0, t.shape[0]])
ax3.set_ylim([fs, 0])
ax3.set_title("Range/Doppler Map")
_ = ax3.annotate(
    f"Peak ({peak_pre_clean[0]},{-(peak_pre_clean[1] - fs // 2):.0f})",
    peak_pre_clean,   
    xytext=(10, -20),
    xycoords="data",
    textcoords="offset pixels",
    arrowprops={"arrowstyle": "wedge"},
)

fig.tight_layout()

In [ ]:
def ambfun(ref, surv):
    return pluto_sdr_pr.processing.fast_ambiguity(samples, fs, ref, surv)

cleaned_surv_signal, _ = pluto_sdr_pr.processing.clean(ref_signal, surv_signal, ambfun)

amb_post_clean = pluto_sdr_pr.processing.fast_ambiguity(
    samples,
    fs,
    ref_signal,
    cleaned_surv_signal,
)
abs_amb_post_clean = np.abs(amb_post_clean)
peak_post_clean = np.unravel_index(np.argmax(np.abs(amb_post_clean)), amb_post_clean.shape)

x = np.arange(amb_pre_clean.shape[0])
y = np.arange(amb_pre_clean.shape[1])
X, Y = np.meshgrid(x, y) 

fig = plt.figure(figsize=(11.69, 8.27))
fig.suptitle("Application of CLEAN Algorithm", fontsize=16)

ax = fig.add_subplot(2, 3, (1, 2))
ax.imshow(abs_amb_pre_clean.T, interpolation="nearest", aspect="auto", cmap="Greys")
ax.set_yticks(np.linspace(0, fs, 9, endpoint=True))
ax.set_yticklabels(
    map(lambda y: f"{-(y - fs // 2):.0f}", ax.get_yticks())
)
ax.set_ylabel("doppler")
ax.set_xlim([0, t.shape[0]])
ax.set_ylim([fs, 0])
ax.set_title("Range/Doppler Map before CLEAN")

ax = fig.add_subplot(2, 3, 3, projection="3d")
ax.plot_surface(X, Y, abs_amb_pre_clean.T / abs_amb_pre_clean.max(), cmap="inferno")
ax.set_yticks(np.linspace(0, fs, 5, endpoint=True))
ax.set_yticklabels(
    map(lambda y: f"{-(y - fs // 2):.0f}", ax.get_yticks())
)
ax.set_xlabel("range")
ax.set_ylabel("doppler")
ax.set_xlim([0, t.shape[0]])
ax.set_ylim([fs, 0])
ax.set_title("Range/Doppler Map before CLEAN (3D)")

ax = fig.add_subplot(2, 3, (4, 5))
ax.imshow(abs_amb_post_clean.T, interpolation="nearest", aspect="auto", cmap="Greys")
ax.set_yticks(np.linspace(0, fs, 9, endpoint=True))
ax.set_yticklabels(
    map(lambda y: f"{-(y - fs // 2):.0f}", ax.get_yticks())
)
ax.set_xlabel("range")
ax.set_ylabel("doppler")
ax.set_xlim([0, t.shape[0]])
ax.set_ylim([fs, 0])
ax.set_title("Range/Doppler Map after CLEAN")
_ = ax.annotate(
    f"Peak ({peak_post_clean[0]},{-(peak_post_clean[1] - fs // 2):.0f})",
    peak_post_clean,   
    xytext=(10, -20),
    xycoords="data",
    textcoords="offset pixels",
    arrowprops={"arrowstyle": "wedge"},
)

ax = fig.add_subplot(2, 3, 6, projection="3d")
ax.plot_surface(X, Y, abs_amb_post_clean.T / abs_amb_post_clean.max(), cmap="inferno")
ax.set_yticks(np.linspace(0, fs, 5, endpoint=True))
ax.set_yticklabels(
    map(lambda y: f"{-(y - fs // 2):.0f}", ax.get_yticks())
)
ax.set_xlabel("range")
ax.set_ylabel("doppler")
ax.set_xlim([0, t.shape[0]])
ax.set_ylim([fs, 0])
ax.set_title("Range/Doppler Map after CLEAN (3D)")

In [ ]:
output_dir = "docu/images/generated"
os.makedirs(output_dir, exist_ok=True)
din_dimensions = (11.69,8.27)

fig = plt.figure(figsize=(din_dimensions[0], 2.5))
ax = fig.add_subplot()
ax.plot(waveform)
ax.set_xlabel("Samples")
ax.set_ylabel("$u(t)$")
ax.set_xlim([0, t.shape[0]])
ax.set_ylim([-1, 1])
ax.grid()
fig.tight_layout()
fig.savefig(os.path.join(output_dir, "clean_base_waveform.svg"))

fig = plt.figure(figsize=fig.get_size_inches())
ax = fig.add_subplot()
ax.plot(ref_waveform)
ax.set_xlabel("Samples")
ax.set_ylabel(r"$y_{ref}(t)$")
ax.set_xlim([0, t.shape[0]])
ax.set_ylim([-1, 1])
ax.grid()
ax.axvspan(0, pulse_on_duration, facecolor='red', alpha=.2, label="Direktsignal")
ax.legend(loc="upper right")
fig.tight_layout()
fig.savefig(os.path.join(output_dir, "clean_ref_waveform.svg"))

fig = plt.figure(figsize=fig.get_size_inches())
ax = fig.add_subplot()
ax.plot(surv_waveform)
ax.set_xlabel("Samples")
ax.set_ylabel(r"$y_{surv}(t)$")
ax.set_xlim([0, t.shape[0]])
ax.set_ylim([-1, 1])
ax.grid()
ax.axvspan(0, pulse_on_duration, facecolor='red', alpha=.2, label="Direktsignal")
ax.axvspan(time_offset, time_offset + pulse_on_duration, facecolor='blue', alpha=.2, label="Zielecho")
ax.legend(loc="upper right")
fig.tight_layout()
fig.savefig(os.path.join(output_dir, "clean_surv_waveform.svg"))

fig = plt.figure(figsize=(din_dimensions[0], 4))
ax = fig.add_subplot()
ax.imshow(abs_amb_pre_clean.T, interpolation="nearest", aspect="auto", cmap="twilight")
ax.set_yticks(np.linspace(0, fs, 9, endpoint=True))
ax.set_yticklabels(
    map(lambda y: f"{-(y - fs // 2):.0f}", ax.get_yticks())
)
ax.set_xlabel("Entfernung")
ax.set_ylabel("Doppler")
ax.set_xlim([0, t.shape[0]])
ax.set_ylim([fs, 0])
_ = ax.annotate(
    f"Direktsignal bei ({peak_pre_clean[0]},{-(peak_pre_clean[1] - fs // 2):.0f})",
    peak_pre_clean,
    xytext=(20, -30),
    xycoords="data",
    textcoords="offset pixels",
    arrowprops={"arrowstyle": "fancy"},
)
_ = ax.annotate(
    f"schwaches Ziel bei ({time_offset},{freq_offset:.0f})",
    (time_offset, fs // 2 - freq_offset),
    xytext=(20, 30),
    xycoords="data",
    textcoords="offset pixels",
    arrowprops={"arrowstyle": "fancy"},
)
fig.tight_layout()
fig.savefig(os.path.join(output_dir, "clean_amb_before_2d.svg"))

fig = plt.figure(figsize=(4, 4))
ax = fig.add_subplot(projection="3d")
ax.plot_surface(X, Y, abs_amb_pre_clean.T / abs_amb_pre_clean.max(), cmap="twilight")
ax.set_yticks(np.linspace(0, fs, 5, endpoint=True))
ax.set_yticklabels(
    map(lambda y: f"{-(y - fs // 2):.0f}", ax.get_yticks())
)
ax.set_xlabel("Entfernung")
ax.set_ylabel("Doppler")
ax.set_xlim([0, t.shape[0]])
ax.set_ylim([fs, 0])
fig.savefig(os.path.join(output_dir, "clean_amb_before_3d.svg"))


fig = plt.figure(figsize=(din_dimensions[0], 4))
ax = fig.add_subplot()
ax.imshow(abs_amb_post_clean.T, interpolation="nearest", aspect="auto", cmap="twilight")
ax.set_yticks(np.linspace(0, fs, 9, endpoint=True))
ax.set_yticklabels(
    map(lambda y: f"{-(y - fs // 2):.0f}", ax.get_yticks())
)
ax.set_xlabel("Entfernung")
ax.set_ylabel("Doppler")
ax.set_xlim([0, t.shape[0]])
ax.set_ylim([fs, 0])
_ = ax.annotate(
    f"Ziel bei ({peak_post_clean[0]},{-(peak_post_clean[1] - fs // 2):.0f})",
    peak_post_clean,
    xytext=(30, 40),
    xycoords="data",
    textcoords="offset pixels",
    arrowprops={"arrowstyle": "wedge"},
)
fig.tight_layout()
fig.savefig(os.path.join(output_dir, "clean_amb_after_2d.svg"))

fig = plt.figure(figsize=(4, 4))
ax = fig.add_subplot(projection="3d")
ax.plot_surface(X, Y, abs_amb_post_clean.T / abs_amb_post_clean.max(), cmap="twilight")
ax.set_yticks(np.linspace(0, fs, 5, endpoint=True))
ax.set_yticklabels(
    map(lambda y: f"{-(y - fs // 2):.0f}", ax.get_yticks())
)
ax.set_xlabel("Entfernung")
ax.set_ylabel("Doppler")
ax.set_xlim([0, t.shape[0]])
ax.set_ylim([fs, 0])
fig.savefig(os.path.join(output_dir, "clean_amb_after_3d.svg"))